In [2]:
from __future__ import print_function

import numpy as np
import random

from matplotlib import pyplot as plt

from selectinf.nbd_lasso import nbd_lasso
from selectinf.Utils.discrete_family import discrete_family
from instance import GGM_instance

from nbd_naive_and_ds import *

In [7]:
# TODO: Add root n to the randomization covariance
# Remark: Not needed (?) anymore since X is now scaled
prec,cov,X = GGM_instance(n=1000,p=10, max_edges=2)
nbd_instance = nbd_lasso.gaussian(X, n_scaled=False)
active_signs_nonrandom = nbd_instance.fit(perturb=np.zeros((10,9)))
active_signs_random = nbd_instance.fit()
print(active_signs_nonrandom.shape)
print(np.abs(active_signs_nonrandom).sum())
print(np.abs(active_signs_random).sum())
print(np.abs(prec != 0).sum() - 10)

Custom perturbation
Sampled perturbation
(10, 9)
2.0
2.0
6


In [6]:
def data_splitting(X, prec, proportion=0.5):
    # Precision matrix is in its original order, not scaled by root n
    # X is also in its original order
    n,p = X.shape
    pi_s = proportion
    subset_select = np.zeros(n, np.bool_)
    subset_select[:int(pi_s * n)] = True
    n1 = subset_select.sum()
    n2 = n - n1
    np.random.shuffle(subset_select)

    # Rescale X_S and X_NS for numerical stability
    X_S = X[subset_select, :] / np.sqrt(n1)
    X_NS = X[~subset_select, :] / np.sqrt(n2)

    nbd_instance = nbd_lasso.gaussian(X_S, n_scaled=True)
    active_signs_nonrandom = nbd_instance.fit(perturb=np.zeros((p,p-1)))
    nonzero = get_nonzero(active_signs_nonrandom)
    # print("Data Splitting |E|:", nonzero.sum())

    # Construct intervals
    if nonzero.sum() > 0:
        # Intervals returned is in its original (unscaled) order
        # intervals, condlDists = conditional_inference(X_NS, nonzero=nonzero)
        intervals = conditional_inference(X_NS, nonzero=nonzero)
        # coverage is upper-triangular
        coverage = get_coverage(nonzero, intervals, prec * n2, n2, p)
        interval_len = 0
        nonzero_count = 0
        for i in range(p):
            for j in range(i+1,p):
                if nonzero[i,j]:
                    interval = intervals[i,j,:]
                    interval_len = interval_len + (interval[1] - interval[0])
                    nonzero_count = nonzero_count + 1
        avg_len = interval_len / nonzero_count
        cov_rate = coverage.sum() / nonzero_count
        return nonzero, intervals, cov_rate, avg_len
    return None, None, None, None

In [8]:
nonzero, intervals, cov_rate, avg_len = data_splitting(X, prec, proportion = 0.67)
print(cov_rate)

Custom perturbation
143.7203195439502
normalizer 9.326974142923583e-16
normalizer == 0 False
log normalizer: -34.60845084044608
normalizer 6.258425627863547e-07
normalizer == 0 False
log normalizer: -14.28416699462342
normalizer 0.09134455914379329
normalizer == 0 False
log normalizer: -2.3931165584815295
normalizer 0.10822693064096982
normalizer == 0 False
log normalizer: -2.2235250466728083
normalizer 0.11600008675481782
normalizer == 0 False
log normalizer: -2.1541643399896917
normalizer 0.109998495575175
normalizer == 0 False
log normalizer: -2.2072885898725643
normalizer 0.10844012582183135
normalizer == 0 False
log normalizer: -2.2215570940624954
normalizer 0.10667968544903993
normalizer == 0 False
log normalizer: -2.2379245282193527
normalizer 0.10704244804496267
normalizer == 0 False
log normalizer: -2.23452981248853
normalizer 0.10753126569641361
normalizer == 0 False
log normalizer: -2.2299736300136774
normalizer 0.10785308958062033
normalizer == 0 False
log normalizer: -2.22

In [5]:
def edge_inference_scipy(j0, k0, S, n, p, Theta_hat=None, var=None, level=0.9, ngrid=10000):
    # n_total: the total data points in data splitting
    #        : the raw dimension of X in naive
    inner_prod = S[j0,k0]

    # Theta_hat: A low dimensional point estimate of theta
    if Theta_hat is None:
        t_j_k = - inner_prod * n
    else:
        t_j_k = Theta_hat[j0,k0] * n
    print(t_j_k)

    def log_det_S_j_k(s_val):
        S_j_k = S
        S_j_k[j0,k0] = s_val
        S_j_k[k0,j0] = s_val
        return np.log(np.abs(np.linalg.det(S_j_k))) * (n-p-1)/2
    def det_S_j_k(s_val):
        return np.exp(log_det_S_j_k(s_val))

    """def get_logdet_normalizer():
        sparse_grid = np.linspace(-2, 2, num=100)
        sparse_log_det = np.zeros((ngrid,))
        for g in range(100):
            sparse_log_det[g] = log_det_S_j_k(sparse_grid[g])
        log_det_normalizer = np.max(sparse_log_det)
        return log_det_normalizer

    ldn = get_logdet_normalizer()
    print("ldn", np.exp(ldn))
    # normalized determinant by multiplying 1/det_S_j_k(0)

    def det_S_j_k_nomalized(s_val, log_normalizer=0):
        return np.exp(log_det_S_j_k(s_val) - np.log(det_S_j_k(0)))"""

    def condl_pdf(t,theta=0):
        return det_S_j_k(t) * np.exp(-theta*t)

    def condl_log_pdf(t,theta=0):
        return log_det_S_j_k(t) - theta * t

    def get_pivot(theta0=0):
        # Normalize the pdf by the maximum over a sparse grid
        def get_pdf_log_normalizer(theta0):
            sparse_grid = np.linspace(-2, 2, num=100)
            sparse_lpdf = np.zeros((ngrid,))
            for g in range(100):
                sparse_lpdf[g] = condl_log_pdf(sparse_grid[g],theta0)
            pdf_log_normalizer = np.max(sparse_lpdf)

            if pdf_log_normalizer > 100:
                pdf_log_normalizer /= 10
            return pdf_log_normalizer

        pdfln = get_pdf_log_normalizer(theta0)
        #print("pdfln", pdfln)
        # Normalized pdf
        def condl_pdf_normalized(t, theta=0):
            return det_S_j_k(t) * np.exp(-theta*t - pdfln)

        grid_lb = -1
        grid_ub = 1
        normalizer = quad(condl_pdf_normalized,
                          grid_lb,
                          grid_ub, args=(theta0,))[0]
        #print(normalizer)

        cdf_upper = quad(condl_pdf_normalized, inner_prod, grid_ub,
                         args=(theta0,))[0] / normalizer

        """stat_grid = np.zeros((1, ngrid))

        stat_grid[0,:] = np.linspace(grid_lb,
                                     grid_ub,
                                     num=ngrid)
        density = np.zeros((ngrid,))
        logdet = np.zeros((ngrid,))
        for g in range(ngrid):
            density[g] = condl_pdf_normalized(stat_grid[0,g], theta0)
            logdet[g] = det_S_j_k(stat_grid[0,g])

        plt.plot(stat_grid[0,:], density)
        plt.figure(2)
        plt.plot(stat_grid[0,:], logdet)"""
        return cdf_upper

    pivot = get_pivot(0)

    def get_pivot_val(x,val=0.99):
        return get_pivot(x) - val

    print("bracket", t_j_k-0.2*n,t_j_k+0.2*n)

    # Construct CI
    margin = (1 - level) / 2
    """root_low = root_scalar(get_pivot_val, bracket=[t_j_k-0.1*n,t_j_k+0.3*n], args=(margin,),
                           method='bisect')
    root_up = root_scalar(get_pivot_val, bracket=[t_j_k-0.3*n,t_j_k+0.1*n], args=(1 - margin,),
                           method='bisect')"""
    root_low = find_root(f=get_pivot, y=margin, lb=t_j_k-0.1*n, ub=t_j_k+0.3*n, tol=1e-6)
    root_up = find_root(f=get_pivot, y=1-margin, lb=t_j_k-0.3*n, ub=t_j_k+0.1*n, tol=1e-6)

    return pivot, root_up, root_low

In [6]:
# Precision matrix is in its original order, not scaled by root n
# X is also in its original order
n,p = X.shape
pi_s = proportion = 0.67
subset_select = np.zeros(n, np.bool_)
subset_select[:int(pi_s * n)] = True
n1 = subset_select.sum()
n2 = n - n1
np.random.shuffle(subset_select)

# Rescale X_S and X_NS for numerical stability
X_S = X[subset_select, :] / np.sqrt(n1)
X_NS = X[~subset_select, :] / np.sqrt(n2)

nbd_instance = nbd_lasso.gaussian(X_S, n_scaled=True)
active_signs_nonrandom = nbd_instance.fit(perturb=np.zeros((p,p-1)))
nonzero = get_nonzero(active_signs_nonrandom)
# print("Data Splitting |E|:", nonzero.sum())

for i in range(p):
    for j in range(i+1,p):
        if nonzero[i,j]:
            print("(i,j) = (",i,",",j,")")
            print("Theta", prec[i,j])

"""# Construct intervals
if nonzero.sum() > 0:
    # Intervals returned is in its original (unscaled) order
    # intervals, condlDists = conditional_inference(X_NS, nonzero=nonzero)
    intervals = conditional_inference(X_NS, nonzero=nonzero)
    # coverage is upper-triangular
    coverage = get_coverage(nonzero, intervals, prec * n2, n2, p)
    interval_len = 0
    nonzero_count = 0
    for i in range(p):
        for j in range(i+1,p):
            if nonzero[i,j]:
                interval = intervals[i,j,:]
                interval_len = interval_len + (interval[1] - interval[0])
                nonzero_count = nonzero_count + 1
    avg_len = interval_len / nonzero_count
    cov_rate = coverage.sum() / nonzero_count"""

Custom perturbation
(i,j) = ( 5 , 9 )
Theta 0.28070579437147164


'# Construct intervals\nif nonzero.sum() > 0:\n    # Intervals returned is in its original (unscaled) order\n    # intervals, condlDists = conditional_inference(X_NS, nonzero=nonzero)\n    intervals = conditional_inference(X_NS, nonzero=nonzero)\n    # coverage is upper-triangular\n    coverage = get_coverage(nonzero, intervals, prec * n2, n2, p)\n    interval_len = 0\n    nonzero_count = 0\n    for i in range(p):\n        for j in range(i+1,p):\n            if nonzero[i,j]:\n                interval = intervals[i,j,:]\n                interval_len = interval_len + (interval[1] - interval[0])\n                nonzero_count = nonzero_count + 1\n    avg_len = interval_len / nonzero_count\n    cov_rate = coverage.sum() / nonzero_count'

In [30]:
S=X_NS.T @ X_NS
theta_hat = np.linalg.inv(S)
pivot, lcb, ucb = edge_inference_scipy(j0=0,k0=5,S=S,n=n2,p=p,Theta_hat=theta_hat)

-92.21428786134462
bracket -158.21428786134462 -26.214287861344616


In [31]:
lcb/n2, ucb/n2

(-0.3712540516773781, -0.17657569741407247)

In [83]:
def print_nonzero_intervals(nonzero, intervals, prec, X):
    # Intervals, prec, X are all in their original scale
    n, p = X.shape
    S = X.T @ X / n

    for i in range(p):
            for j in range(i+1,p):
                if nonzero[i,j]:
                    print("(",i,",",j,")", "selected")
                    print("Theta", "(",i,",",j,")", "interval:", intervals[i,j,:])
                    print("Theta", "(",i,",",j,")", prec[i,j])
                    print("S/n", "(",i,",",j,")", S[i,j])
print_nonzero_intervals(nonzero, intervals, prec, X)

( 3 , 7 ) selected
Theta ( 3 , 7 ) interval: [0.20787315 0.38201774]
Theta ( 3 , 7 ) 0.0
S/n ( 3 , 7 ) -0.002432089772410599
